# 08.4.线性回归的简洁实现
通过使用深度学习框架来简洁地实现线性回归模型生成数据集

In [2]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

调用框架中现有的API来读取数据

In [4]:
def load_array(data_arrays, batch_size, is_train = True):
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)#*表示接受任意多个参数并将其放在一个元组中
    return data.DataLoader(dataset, batch_size, shuffle = is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)
next(iter(data_iter))#通过next函数得到一个X和一个y

[tensor([[-3.9289e-01, -1.0004e+00],
         [ 5.6753e-01,  6.1301e-01],
         [-1.0358e+00,  5.2048e-04],
         [ 6.8224e-01, -1.2040e+00],
         [ 3.1840e-01, -1.8332e+00],
         [ 1.1220e+00, -2.2689e-01],
         [ 3.2423e-01,  2.7913e-01],
         [-3.6689e-01, -1.2123e+00],
         [ 2.0974e-01,  3.2510e-01],
         [-1.2308e+00,  6.1364e-01]]),
 tensor([[ 6.8274],
         [ 3.2543],
         [ 2.1131],
         [ 9.6555],
         [11.0763],
         [ 7.2036],
         [ 3.9030],
         [ 7.6003],
         [ 3.5112],
         [-0.3437]])]

使用框架的预定义好的层

In [12]:
# `nn`是神经网络的缩写
from torch import nn
"""
nn.Sequential创建了一个list of layers
输入是2维 输出是1维
"""
net = nn.Sequential(nn.Linear(2, 1))


初始化模型参数

In [10]:
#net[0]访问到nn.Linear(2, 1)
net[0].weight.data.normal_(0,0.01)#.weight访问到w，.data是真实data，normal_会使用正态分布替换掉data的值，两个参数是均值和方差
net[0].bias.data.fill_(0)#bais是偏差1

tensor([0.])

计算均方误差使用的是`MSELoss`类，也称为平方范数

In [17]:
loss = nn.MSELoss()

实例化`SGD`实例

In [20]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)
#net.parameters()获得所有参数包括`w`和`b` lr是学习率

In [40]:
num_epochs = 3
for epoch in range(num_epochs):
    for X,y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000107
epoch 2, loss 0.000109
epoch 3, loss 0.000107
